# Qwen-Image-Edit - Pruebas Locales

Este notebook te permite probar Qwen-Image-Edit localmente sin desplegar en RunPod.

## Requisitos:
- GPU con al menos 20GB VRAM
- Docker con soporte NVIDIA
- Contenedor ejecutándose localmente


In [ ]:
# Verificar GPU disponible
import torch
import subprocess

print("=== VERIFICACIÓN DE HARDWARE ===")
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"VRAM libre: {torch.cuda.memory_reserved(0) / 1024**3:.1f} GB")

# Verificar memoria del sistema
try:
    result = subprocess.run(['nvidia-smi', '--query-gpu=memory.total,memory.used,memory.free', '--format=csv,noheader,nounits'], 
                          capture_output=True, text=True)
    if result.returncode == 0:
        total, used, free = result.stdout.strip().split(', ')
        print(f"\nNVIDIA-SMI:")
        print(f"  Total: {int(total)} MB")
        print(f"  Usado: {int(used)} MB")
        print(f"  Libre: {int(free)} MB")
except:
    print("No se pudo obtener información de nvidia-smi")


In [ ]:
# Cargar Qwen-Image-Edit Pipeline
import time
from diffusers import QwenImageEditPipeline
from PIL import Image
import matplotlib.pyplot as plt

print("=== CARGANDO QWEN-IMAGE-EDIT ===")
start_time = time.time()

try:
    # Cargar el pipeline
    pipeline = QwenImageEditPipeline.from_pretrained(
        "Qwen/Qwen-Image-Edit",
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        use_safetensors=True
    ).to("cuda")
    
    # Optimizaciones de memoria
    pipeline.enable_memory_efficient_attention()
    pipeline.enable_model_cpu_offload()
    
    load_time = time.time() - start_time
    print(f"✅ Pipeline cargado exitosamente en {load_time:.2f} segundos")
    
    # Verificar memoria después de la carga
    if torch.cuda.is_available():
        print(f"VRAM usada: {torch.cuda.memory_allocated(0) / 1024**3:.1f} GB")
        print(f"VRAM reservada: {torch.cuda.memory_reserved(0) / 1024**3:.1f} GB")
        
except Exception as e:
    print(f"❌ Error cargando el pipeline: {str(e)}")
    print("Verifica que tienes suficiente VRAM (mínimo 20GB)")
